In [1]:
# Set up logging

import logging

logging.basicConfig(
    format="[%(asctime)s %(levelname)s %(threadName)s %(name)s:%(funcName)s:%(lineno)s] %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S%z",
)
logging.getLogger("council").setLevel(logging.DEBUG)

In [2]:
# Council imports

from council.runners import Budget
from council.contexts import AgentContext, ChatHistory
from council.agents import Agent
from council.chains import Chain
from council.llm.openai_llm import OpenAILLM

from skills import SectionWriterSkill, OutlineWriterSkill
from controller import WritingAssistantController
from evaluator import BasicEvaluatorWithSource

In [3]:
# Read environment variables and initialize OpenAI LLM instance

import dotenv
dotenv.load_dotenv()
openai_llm = OpenAILLM.from_env()

[2023-08-08 19:24:25-0400 WARNING MainThread council.llm.openai_token_counter:from_model:139] gpt-4 may change over time. Returning num tokens assuming gpt-4-0613.


In [4]:
# Create Skills

outline_skill = OutlineWriterSkill(openai_llm)
writing_skill = SectionWriterSkill(openai_llm)

In [5]:
# Create Chains

outline_chain = Chain(
    name="Outline Writer",
    description="Write or revise the outline (i.e. section headers) of a research article in markdown format. Only use this chain when the ARTICLE OUTLINE is missing or incomplete.",
    runners=[outline_skill]
)

writer_chain = Chain(
    name="Article Writer",
    description="Write or revise specific section bodies of a research article in markdown format. Use this chain to write the main research article content.",
    runners=[writing_skill]
)

In [6]:
# Create Controller

controller = WritingAssistantController(
    openai_llm,
    top_k_execution_plan=5
)

In [7]:
# Initialize Agent

chat_history = ChatHistory()
run_context = AgentContext(chat_history)
agent = Agent(controller, [outline_chain, writer_chain], BasicEvaluatorWithSource())

In [8]:
# Run Agent

chat_history.add_user_message("Write a detailed research article about the history of box manufacturing.")
result = agent.execute(run_context, Budget(1800))

[2023-08-08 19:24:25-0400 INFO MainThread council.agents.agent:execute:60] message="agent execution started"
[2023-08-08 19:24:25-0400 INFO MainThread council.agents.agent:execute:62] message="agent iteration started" iteration="1"
[2023-08-08 19:24:25-0400 DEBUG MainThread council.llm.llm_base:post_chat_request:56] message="starting execution of llm request"
[2023-08-08 19:24:25-0400 DEBUG MainThread council.llm.openai_chat_completions_llm:_post_request:154] message="Sending chat GPT completions request" payload="{'temperature': 0.1, 'n': 1, 'model': 'gpt-4', 'messages': [{'role': 'system', 'content': 'You are the Controller module for an AI assistant built to write and revise research articles.'}, {'role': 'user', 'content': "\n        # Task Description\n        Your task is to decide how best to write or revise the ARTICLE. Considering the ARTICLE OUTLINE, ARTICLE, and the CONVERSATION HISTORY,\n        use your avaiable CHAINS to decide what steps to take next. You are not respons

In [9]:
print(result.messages[-1].message.message)

# Introduction

Box manufacturing has a rich and varied history that dates back centuries. This industry has played a crucial role in the development of trade, commerce, and everyday life, providing a simple yet effective solution for storage, transportation, and packaging of goods. The evolution of box manufacturing is a testament to human ingenuity and the relentless pursuit of efficiency and convenience. 

The importance of studying the history of box manufacturing lies in its profound impact on human civilization. From the ancient Egyptian wooden chests used for storing personal belongings to the modern-day cardboard boxes used for shipping goods around the world, boxes have been an integral part of our lives. They have facilitated trade, enabled exploration, and made it possible to store and transport goods safely and efficiently. The history of box manufacturing is, in many ways, the history of human progress.

This research aims to provide a comprehensive overview of the history

In [10]:
chat_history.add_user_message("Can you please add a section (including subsections) on boxes in popular culture?")
result = agent.execute(run_context, Budget(600))

[2023-08-08 19:31:16-0400 INFO MainThread council.agents.agent:execute:60] message="agent execution started"
[2023-08-08 19:31:16-0400 INFO MainThread council.agents.agent:execute:62] message="agent iteration started" iteration="3"
[2023-08-08 19:31:16-0400 DEBUG MainThread council.llm.llm_base:post_chat_request:56] message="starting execution of llm request"
[2023-08-08 19:31:16-0400 DEBUG MainThread council.llm.openai_chat_completions_llm:_post_request:154] message="Sending chat GPT completions request" payload="{'temperature': 0.1, 'n': 1, 'model': 'gpt-4', 'messages': [{'role': 'system', 'content': 'You are the Controller module for an AI assistant built to write and revise research articles.'}, {'role': 'user', 'content': "\n        # Task Description\n        Your task is to decide how best to write or revise the ARTICLE. Considering the ARTICLE OUTLINE, ARTICLE, and the CONVERSATION HISTORY,\n        use your avaiable CHAINS to decide what steps to take next. You are not respons

In [11]:
print(result.messages[-1].message.message)

# Introduction

Box manufacturing has a rich and varied history that dates back centuries. This industry has played a crucial role in the development of trade, commerce, and everyday life, providing a simple yet effective solution for storage, transportation, and packaging of goods. The evolution of box manufacturing is a testament to human ingenuity and the relentless pursuit of efficiency and convenience. 

The importance of studying the history of box manufacturing lies in its profound impact on human civilization. From the ancient Egyptian wooden chests used for storing personal belongings to the modern-day cardboard boxes used for shipping goods around the world, boxes have been an integral part of our lives. They have facilitated trade, enabled exploration, and made it possible to store and transport goods safely and efficiently. The history of box manufacturing is, in many ways, the history of human progress.

This research aims to provide a comprehensive overview of the history